In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.initializers import  RandomNormal
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import PIL

In [2]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))


Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Preprocessing Dataset

In [3]:
img_width, img_height = 48, 48
batch_size = 32

# Define the data directories
train_data_dir = 'images/train'
validation_data_dir = 'images/validation'

In [4]:
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1] by scaling by 255
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2,  
    zoom_range=0.2,  
    vertical_flip=False,
    horizontal_flip=True
)

train_gen = data_gen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
)

val_gen = data_gen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 7178 images belonging to 7 classes.
Found 3668 images belonging to 7 classes.


## Model

In [5]:
# Constructing CNN structure
model = Sequential()

# 1st convolution layer
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', input_shape=(img_width, img_height, 3), bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', input_shape=(img_width, img_height, 3), bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))

# 3rd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))
          
# 5th convolution layer
model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))

# 7th convolution layer
model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
# Fully connected layers
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [6]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 64)        3

In [7]:
# Train the model
epochs = 100
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    validation_data=val_gen,
    validation_steps=val_gen.samples // batch_size,
    epochs=epochs)

model.save("cnnModel.h5")

Epoch 1/100
224/224 [==============================] - 41s 164ms/step - loss: 2.7620 - accuracy: 0.1919 - val_loss: 1.8154 - val_accuracy: 0.2593
Epoch 2/100
224/224 [==============================] - 11s 51ms/step - loss: 1.8594 - accuracy: 0.2112 - val_loss: 1.9707 - val_accuracy: 0.2612
Epoch 3/100
224/224 [==============================] - 12s 52ms/step - loss: 1.8381 - accuracy: 0.2267 - val_loss: 1.9679 - val_accuracy: 0.2664
Epoch 4/100
224/224 [==============================] - 12s 51ms/step - loss: 1.8361 - accuracy: 0.2291 - val_loss: 1.9646 - val_accuracy: 0.2522
Epoch 5/100
224/224 [==============================] - 12s 51ms/step - loss: 1.8256 - accuracy: 0.2393 - val_loss: 1.9623 - val_accuracy: 0.2596
Epoch 6/100
224/224 [==============================] - 11s 51ms/step - loss: 1.8248 - accuracy: 0.2319 - val_loss: 1.9667 - val_accuracy: 0.2527
Epoch 7/100
224/224 [==============================] - 11s 51ms/step - loss: 1.8238 - accuracy: 0.2372 - val_loss: 2.1228 - val_a

In [8]:
model2 = load_model('cnnModel.h5')

In [10]:
scores = model2.evaluate(train_gen, batch_size=1024)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

225/225 [==============================] - 9s 37ms/step - loss: 1.6961 - accuracy: 0.4638
Loss: 1.6960623264312744
Accuracy: 0.46377819776535034


In [12]:
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(val_gen.classes, y_pred))
print('Classification Report')
target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print(classification_report(val_gen.classes, y_pred, target_names=target_names))

115/115 [==============================] - 4s 35ms/step
Confusion Matrix
[[  3   0  11  97  80 124  68]
 [  0   0   0  15   5  18   8]
 [  6   0  17 109 114 154  69]
 [ 14   0  24 234 246 317 144]
 [  7   0  24 175 148 236 103]
 [ 10   0  24 159 168 214  97]
 [  7   0  18 110 102 133  56]]
Classification Report
              precision    recall  f1-score   support

       Angry       0.06      0.01      0.01       383
     Disgust       0.00      0.00      0.00        46
        Fear       0.14      0.04      0.06       469
       Happy       0.26      0.24      0.25       979
         Sad       0.17      0.21      0.19       693
    Surprise       0.18      0.32      0.23       672
     Neutral       0.10      0.13      0.12       426

    accuracy                           0.18      3668
   macro avg       0.13      0.14      0.12      3668
weighted avg       0.17      0.18      0.17      3668



c:\Users\techn\anaconda3\envs\gpu_fix\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\techn\anaconda3\envs\gpu_fix\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\techn\anaconda3\envs\gpu_fix\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
